## Richardson Extrapolation and and Spline Derivative

In [1]:
using LinearAlgebra
# using Plots
using PyCall
using PyPlot
using DataFrames
using CSV
using Symbolics
using Integrals
# plotlyjs()
matplotlib.use("TkAgg")

### Richardson Extrapolation
Richardson extrapolation is done with two iterations to find a accurate derivative value
1. F is defined as: $F = \frac{1}{h}(y(x_0+h/2)-y(x_0-h/2))$
2. First iteration: $\tilde F  = \frac{4}{3}(F(h/2)-F(h))$
2. Second iteration: $\frac{16}{15}(\tilde F(h/2)-\tilde F(h))$

In [5]:
function F_Richardson(y::Function,x₀,h)
    1/h*(y(x₀+h/2)-y(x₀-h/2))
end
function F̃_Richardson(y::Function,x₀,h)
    F = (h)->F_Richardson(y,x₀,h)
    4/3*F(h/2) - 1/3*F(h)
end

function Richardson_2iter(y::Function,x₀,h)
    F̃ = (h)->F̃_Richardson(y,x₀,h)
    16/15*F̃(h/2) - 1/15*F̃(h)
end

Richardson_2iter (generic function with 1 method)

#### Defining central difference method for Comparison

In [6]:
function Central_Difference(y::Function,x₀,h)
    (-y(x₀+2h) + 8y(x₀+h) - 8y(x₀-h) + y(x₀-2h))/(12h)
end

Central_Difference (generic function with 1 method)

In [12]:
print("Derivative of tan⁻¹(x) at x = 0 by Richardson Extrapolation: ", Richardson_2iter(atan,0,0.1), "\n")
print("Error as compared to true value of derivative: ", abs(Richardson_2iter(atan,0,0.1) - 1), "\n")

Derivative of tan⁻¹(x) at x = 0 by Richardson Extrapolation: 0.9999999999652116
Error as compared to true value of derivative: 3.478839438741943e-11


In [13]:
print("Derivative of tan⁻¹(x) at x = 0 by Central Differences: ", Central_Difference(atan,0,0.1), "\n")
print("Error as compared to true value of derivative:", abs(Central_Difference(atan,0,0.1) - 1), "\n")

Derivative of tan⁻¹(x) at x = 0 by Central Differences: 0.9999227667990257
Error as compared to true value of derivative:7.723320097430264e-5


Richardson extrapolation is approximately $10^6$ times more accurate (for this problem)

### Spline Derivative
Here, we get the spline coefficients $a_i, b_i, c_i $ and $ d_i$ <br>
Spline derivative is defined as $ y(x)=3a_i (x-x_i)^2+2b_i (x-x_i)+c_i $

In [16]:
function spline_coeff(f::Function,n,lim;verbose=false)
    Y = f.(range(lim[1],lim[2],length=n+1))
    h= Y[2]-Y[1]

    if verbose
        print(h,' ',n ,'\n')
        
    end

    # S=0 AND Sₙ=0
    A = Tridiagonal(repeat([h],n-2),repeat([2h],n-1),repeat([h],n-2))

    
    if verbose
        print(size(A),'\n')
        print(size(Y),'\n')
        
    end
    S = A\(Y[2:end-1])
    
    S = [0;S;0]
    # Y = [Y;f₁(lim[2])]
    if verbose
        print(size(S),'\n')
        print(size(Y),'\n')
        
    end
    aᵢ=zeros(n)
    bᵢ=zeros(n)
    cᵢ=zeros(n)
    dᵢ=zeros(n)
    for i in 1:(n)
        aᵢ[i] = (S[i+1]-S[i])/(6h)
        bᵢ[i] = S[i]/2
        cᵢ[i] = (Y[i+1]-Y[i])/h - (2h*S[i]+h*S[i+1])/6
        dᵢ[i] = Y[i]
    end
    return aᵢ,bᵢ,cᵢ,dᵢ,h
end

function spline_derivative(y::Function,x₀,h)

    aᵢ,bᵢ,cᵢ,dᵢ,h = spline_coeff(y,3,[x₀-h,x₀+h])

    3aᵢ[1]*h^2 + 2bᵢ[1]*h + cᵢ[1]
end

spline_derivative (generic function with 1 method)

In [ ]:
print("Derivative of tan⁻¹(x) at x = 0 by Splines: ", spline_derivative(atan,0,0.1), "\n")
print("Error as compared to true value of derivative: ", abs(spline_derivative(atan,0,0.1) - 1), "\n")
#Pretty high error

Derivative of tan⁻¹(x) at x = 0 by Splines: 0.9888930013739177
Error as compared to true value of derivative: 0.011106998626082332
